<a href="https://colab.research.google.com/github/smomb/BaThesis_ImperviousCD/blob/main/BachelorImperviosCD_Holub.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Importing libraries/modules.

import ee
import geemap.core
import geemap
import os

!pip install folium geopandas pandas matplotlib
import folium
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import pprint
import numpy as np

In [ ]:
#GEE authentication and initialization

ee.Authenticate()
ee.Initialize(project='bscthesischangedetection')

In [ ]:
# Loading study area shape.

catchment = ee.FeatureCollection('projects/bscthesischangedetection/assets/catchment_wienfluss')


In [ ]:
# Fetching and visualize data.

map_center = catchment.geometry().centroid().coordinates().getInfo()[::-1]
m = geemap.Map(center=map_center, zoom=9)
m.addLayer(catchment, {}, 'Catchment')

# Visualization parameters
vis_tm = {'bands': ['SR_B3','SR_B2','SR_B1'], 'min': 0, 'max': 12000}
vis_ol = {'bands': ['SR_B4','SR_B3','SR_B2'], 'min': 0, 'max': 12000}

#Dataset 1: 1984 (Landsat 5 TM)
image1_1 = ee.Image('LANDSAT/LT05/C02/T1_L2/LT05_190026_19840805')
image1_1_clipped = image1_1.clip(catchment)
m.addLayer(image1_1_clipped, vis_tm, 'scene_1_1984')

#Dataset 2: 1997 (Landsat 5 TM)
image1_2 = ee.Image('LANDSAT/LT05/C02/T1_L2/LT05_190026_19970825')
image1_2_clipped = image1_2.clip(catchment)
m.addLayer(image1_2_clipped, vis_tm, 'scene_1_1997')

#Dataset 3: 2013 (Landsat 8 OLI)
image1_3 = ee.Image('LANDSAT/LC08/C02/T1_L2/LC08_190026_20130805')
image1_3_clipped = image1_3.clip(catchment)
m.addLayer(image1_3_clipped, vis_ol, 'scene_1_2013')

#Dataset 4: 2024 (Landsat 9 OLI-2)
image1_4 = ee.Image('LANDSAT/LC09/C02/T1_L2/LC09_190026_20240811')
image1_4_clipped = image1_4.clip(catchment)
m.addLayer(image1_4_clipped, vis_ol, 'scene_1_2024')
m


In [ ]:
# Fetch and print metadata for each selected and clipped image.

image1_metadata = {image1_1_clipped.get('DATE_ACQUIRED').getInfo(),
                   image1_1_clipped.get('CLOUD_COVER').getInfo(),
                   image1_1_clipped.get('system:id').getInfo()}
pprint.pprint(image1_metadata)

image2_metadata = {image1_2_clipped.get('DATE_ACQUIRED').getInfo(),
                   image1_2_clipped.get('CLOUD_COVER').getInfo(),
                   image1_2_clipped.get('system:id').getInfo()}
pprint.pprint(image2_metadata)

image3_metadata = {image1_3_clipped.get('DATE_ACQUIRED').getInfo(),
                   image1_3_clipped.get('CLOUD_COVER').getInfo(),
                   image1_3_clipped.get('system:id').getInfo()}
pprint.pprint(image3_metadata)

image4_metadata = {image1_4_clipped.get('DATE_ACQUIRED').getInfo(),
                   image1_4_clipped.get('CLOUD_COVER').getInfo(),
                   image1_4_clipped.get('system:id').getInfo()}
pprint.pprint(image4_metadata)

In [ ]:
#Apply Normalized Difference Water Index. Examine histogram for manual thresholding of water features from non-water features.

#Using Modified Normalized Difference Water Index (mNDWI) instead of NDWI due to the large degree of urbanization in the study area.

# 1984 (Landsat 5 TM): Green = SR_B2, SWIR1 = SR_B5
mndwi_1 = image1_1_clipped.expression(
    '(g - s) / (g + s)',
    {'g': image1_1_clipped.select('SR_B2'),
        's': image1_1_clipped.select('SR_B5')
    }).rename('mndwi')

# 1997 (Landsat 5 TM): same as previous
mndwi_2 = image1_2_clipped.expression(
    '(g - s) / (g + s)',
    {'g': image1_2_clipped.select('SR_B2'),
        's': image1_2_clipped.select('SR_B5')
    }).rename('mndwi')

# 2013 (Landsat 8): Green = SR_B3, SWIR1 = SR_B6
mndwi_3 = image1_3_clipped.expression(
    '(g - s) / (g + s)',
    {'g': image1_3_clipped.select('SR_B3'),
        's': image1_3_clipped.select('SR_B6')
    }).rename('mndwi')

# 2024 (Landsat 9): same bands as Landsat 8
mndwi_4 = image1_4_clipped.expression(
    '(g - s) / (g + s)',
    {'g': image1_4_clipped.select('SR_B3'),
        's': image1_4_clipped.select('SR_B6')
    }).rename('mndwi')

In [ ]:
# Get the mndwi values as a NumPy array and then access the values using the band name.
mndwi_1_np = np.array(mndwi_1.reduceRegion(
    reducer=ee.Reducer.toList(),
    geometry=mndwi_1.geometry(),
    scale=30,
    maxPixels=1e13
).getInfo()['mndwi'])

# Creating a histogram using matplotlib.pyplot
plt.hist(mndwi_1_np, bins=50)
plt.title("mNDWI Histogram - Image 1 1984")
plt.xlabel("mNDWI Value")
plt.ylabel("Pixel Count")
plt.show()

In [ ]:
# Repeating this for all images.
mndwi_2_np = np.array(mndwi_2.reduceRegion(
    reducer=ee.Reducer.toList(),
    geometry=mndwi_2.geometry(),
    scale=30,
    maxPixels=1e13
).getInfo()['mndwi'])
plt.title("mNDWI Histogram - Image 2 1997")
plt.xlabel("mNDWI Value")
plt.ylabel("Pixel Count")
plt.show()

In [ ]:
mndwi_3_np = np.array(mndwi_3.reduceRegion(
    reducer=ee.Reducer.toList(),
    geometry=mndwi_3.geometry(),
    scale=30,
    maxPixels=1e13
).getInfo()['mndwi'])
plt.hist(mndwi_3_np, bins=50)
plt.title("mNDWI Histogram - Image 3 2013")
plt.xlabel("mNDWI Value")
plt.ylabel("Pixel Count")
plt.show()

In [ ]:
mndwi_4_np = np.array(mndwi_4.reduceRegion(
    reducer=ee.Reducer.toList(),
    geometry=mndwi_4.geometry(),
    scale=30,
    maxPixels=1e13
).getInfo()['mndwi'])
plt.hist(mndwi_4_np, bins=50)
plt.title("mNDWI Histogram - 2024")
plt.xlabel("mNDWI Value")
plt.ylabel("Pixel Count")
plt.show()

In [ ]:
# Manual threshholding. Assuming 0.0 for all datasets due to the small amount of values over 0, likely being consistent with actual water areas present in the images.
mndwi_threshhold_1 = 0.0
mndwi_threshhold_2 = 0.0
mndwi_threshhold_3 = 0.0
mndwi_threshhold_4 = 0.0

# Masking the water pixels with the previously defined threshhold.
water_mask_1 = mndwi_1.gt(mndwi_threshhold_1)
water_mask_2 = mndwi_2.gt(mndwi_threshhold_2)
water_mask_3 = mndwi_3.gt(mndwi_threshhold_3)
water_mask_4 = mndwi_4.gt(mndwi_threshhold_4)

image1_1_clipped = image1_1_clipped.updateMask(water_mask_1.Not())
image1_2_clipped = image1_2_clipped.updateMask(water_mask_2.Not())
image1_3_clipped = image1_3_clipped.updateMask(water_mask_3.Not())
image1_4_clipped = image1_4_clipped.updateMask(water_mask_4.Not())


In [ ]:
# Cloud masking and Quality Assurance-based pixel masking to eliminate clouds, cloud shadows, and unused pixels from the mosaics, following Landsat documentation.

qa = image1_1_clipped.select('QA_PIXEL')
cloud_mask = qa.bitwiseAnd(1 << 0).eq(0)   # bit 0: Fill, keep if 0 (not fill)
cloud_mask = cloud_mask.And(qa.bitwiseAnd(1 << 1).eq(0))  # bit 1: Dilated Cloud
cloud_mask = cloud_mask.And(qa.bitwiseAnd(1 << 2).eq(0))  # bit 2: Unused Pixel
cloud_mask = cloud_mask.And(qa.bitwiseAnd(1 << 3).eq(0))  # bit 3: Cloud
cloud_mask = cloud_mask.And(qa.bitwiseAnd(1 << 4).eq(0))  # bit 4: Cloud Shadow
image1_1_clipped = image1_1_clipped.updateMask(cloud_mask)

qa = image1_2_clipped.select('QA_PIXEL')
cloud_mask = qa.bitwiseAnd(1 << 0).eq(0)   # bit 0: Fill, keep if 0 (not fill)
cloud_mask = cloud_mask.And(qa.bitwiseAnd(1 << 1).eq(0))  # bit 1: Dilated Cloud
cloud_mask = cloud_mask.And(qa.bitwiseAnd(1 << 2).eq(0))  # bit 2: Unused Pixel
cloud_mask = cloud_mask.And(qa.bitwiseAnd(1 << 3).eq(0))  # bit 3: Cloud
cloud_mask = cloud_mask.And(qa.bitwiseAnd(1 << 4).eq(0))  # bit 4: Cloud Shadow
image1_2_clipped = image1_2_clipped.updateMask(cloud_mask)

qa = image1_3_clipped.select('QA_PIXEL')
cloud_mask = qa.bitwiseAnd(1 << 0).eq(0)   # bit 0: Fill, keep if 0 (not fill)
cloud_mask = cloud_mask.And(qa.bitwiseAnd(1 << 1).eq(0))  # bit 1: Dilated Cloud
cloud_mask = cloud_mask.And(qa.bitwiseAnd(1 << 2).eq(0))  # bit 2: Cirrus
cloud_mask = cloud_mask.And(qa.bitwiseAnd(1 << 3).eq(0))  # bit 3: Cloud
cloud_mask = cloud_mask.And(qa.bitwiseAnd(1 << 4).eq(0))  # bit 4: Cloud Shadow
image1_3_clipped = image1_3_clipped.updateMask(cloud_mask)

qa = image1_4_clipped.select('QA_PIXEL')
cloud_mask = qa.bitwiseAnd(1 << 0).eq(0)   # bit 0: Fill, keep if 0 (not fill)
cloud_mask = cloud_mask.And(qa.bitwiseAnd(1 << 1).eq(0))  # bit 1: Dilated Cloud
cloud_mask = cloud_mask.And(qa.bitwiseAnd(1 << 2).eq(0))  # bit 2: Cirrus
cloud_mask = cloud_mask.And(qa.bitwiseAnd(1 << 3).eq(0))  # bit 3: Cloud
cloud_mask = cloud_mask.And(qa.bitwiseAnd(1 << 4).eq(0))  # bit 4: Cloud Shadow
image1_4_clipped = image1_4_clipped.updateMask(cloud_mask)

In [ ]:
# Defining the masks of all four images as variables in order to combine them into a global mask.
mask1 = image1_1_clipped.mask()
mask2 = image1_2_clipped.mask()
mask3 = image1_3_clipped.mask()
mask4 = image1_4_clipped.mask()

# Global mask
global_mask = mask1.And(mask2).And(mask3).And(mask4)

# Applying that global mask to each scene to ensure consistent comparison across all images.

image1_1_clipped = image1_1_clipped.updateMask(global_mask)
image1_2_clipped = image1_2_clipped.updateMask(global_mask)
image1_3_clipped = image1_3_clipped.updateMask(global_mask)
image1_4_clipped = image1_4_clipped.updateMask(global_mask)

In [ ]:
# Application of a scale factor of 0.0000275 and an offset of −0.2, provided by the Landsat documentation and the normalization of all values on a conceptual scale spanning from 0 to 1.

optical_bands = image1_1_clipped.select('SR_B.*').multiply(0.0000275).add(-0.2)

image1_1_clipped = image1_1_clipped.addBands(optical_bands, overwrite=True)

optical_bands = image1_2_clipped.select('SR_B.*').multiply(0.0000275).add(-0.2)
image1_2_clipped = image1_2_clipped.addBands(optical_bands, overwrite=True)

optical_bands = image1_3_clipped.select('SR_B.*').multiply(0.0000275).add(-0.2)
image1_3_clipped = image1_3_clipped.addBands(optical_bands, overwrite=True)

optical_bands = image1_4_clipped.select('SR_B.*').multiply(0.0000275).add(-0.2)
image1_4_clipped = image1_4_clipped.addBands(optical_bands, overwrite=True)

# Clamping reflectance bands to [0, 1] to remove any slight negatives or >1 values

band_names = image1_1_clipped.bandNames().remove('QA_PIXEL').remove('QA_RADSAT')
image1 = image1_1_clipped.select(band_names).clamp(0.0, 1.0).addBands(image1_1.select(['QA_PIXEL', 'QA_RADSAT']), overwrite=True)

band_names = image1_2_clipped.bandNames().remove('QA_PIXEL').remove('QA_RADSAT')
image2 = image1_2_clipped.select(band_names).clamp(0.0, 1.0).addBands(image1_2.select(['QA_PIXEL', 'QA_RADSAT']), overwrite=True)

band_names = image1_3_clipped.bandNames().remove('QA_PIXEL').remove('QA_RADSAT')
image3 = image1_3_clipped.select(band_names).clamp(0.0, 1.0).addBands(image1_3.select(['QA_PIXEL', 'QA_RADSAT']), overwrite=True)

band_names = image1_4_clipped.bandNames().remove('QA_PIXEL').remove('QA_RADSAT')
image4 = image1_4_clipped.select(band_names).clamp(0.0, 1.0).addBands(image1_4.select(['QA_PIXEL', 'QA_RADSAT']), overwrite=True)


In [ ]:
def classifyImperviousSVM(image,
                          catchment,
                          bands,
                          year_label,
                          sample_points,
                          seed=999,
                          b1_max=None,
                          gamma_values=None,
                          cost_values=None):
    """
    Train an SVM classifier on 'image' for impervious/v.s. pervious surfaces,
    using the GISD30 'b1' truth data.

    Args:
      image        : ee.Image to classify
      catchment    : ee.Feature defining the study area
      bands        : list of band names on image to use
      year_label   : string, for map & print labels (e.g. '2013')
      sample_points: int, how many random reference points to draw
      seed         : int, random seed for consistency between runs
      b1_max       : int, value 1 for 1984, 4 for 1997, 7 for 2013, 8/None for 2024
      gamma_values : list of floats (grid for SVM γ).  Defaults 0.1–1.0
      cost_values  : list of ints   (grid for SVM C).  Defaults 1–10

    Returns:
      dict with keys:
        'classifier'      : the final ee.Classifier
        'classifiedImage' : ee.Image classification
        'accuracy'        : float overall accuracy
        'areas_m2'        : {'impervious':…, 'pervious':…, 'total':…}
    """

    # 1) Load truth data.
    gisd30 = ee.Image("projects/sat-io/open-datasets/GISD30_1985_2020") \
                .select("b1") \
                .unmask(0)

    # 2) Define the valid sampling region.
    # Reducing the sampling region to areas  not affected by any masking to prevent sample points with no data.
    validMask = (image.select(bands)
                      .mask()
                      .reduce(ee.Reducer.min())
                      .toUint8())
    validGeom = validMask.reduceToVectors(
        geometry=catchment.geometry(),
        scale=30,
        geometryType='polygon',
        eightConnected=False,
        labelProperty='mask'
    ).geometry()

    # 3) Generate random reference points. Multiplying the amount of sample points as defined by the arg sample_points to account for losses from undersampling.
    ref_pts = ee.FeatureCollection.randomPoints(
        region=validGeom,
        points=3 * sample_points,
        seed=seed
    )

    n_rand = ref_pts.size().getInfo()
    print(f"{year_label} randomPoints requested = {sample_points}, actual = {n_rand}")

    # 4) Sample GISD30 at those points.
    gisd_samp = gisd30.sampleRegions(
        collection=ref_pts,
        properties=[],
        scale=30,
        geometries=True
    )

    # 5) Filtering of irrelevant b1 codes if applicable.
    if b1_max is not None:
        gisd_samp = gisd_samp.filter(ee.Filter.lte('b1', b1_max))

    n_gisd = gisd_samp.size().getInfo()
    print(f"{year_label} after sampling GISD30 = {n_gisd}")

    # 6) Turning GISD30 temporal information-including impervious codes into "1" based on image year; "CID" = 1 if b1>0, else 0.
    def _setCID(feat):
        return feat.set('CID', ee.Number(feat.get('b1')).gt(0))
    gisd_pts = gisd_samp.map(_setCID)

    # 7) Adding a random column, sampling the image bands at those truth points.
    gisd_pts = gisd_pts.randomColumn('random', seed=seed)
    samples = image.select(bands).sampleRegions(
        collection=gisd_pts,
        properties=['CID','random'],
        scale=30,
        geometries=True
    )

    # 8) Creating train/validation dataset.
    # 8a) Balancing classes.
    perv = samples.filter(ee.Filter.eq('CID', 0))
    imp  = samples.filter(ee.Filter.eq('CID', 1))
    n_perv = perv.size().getInfo()
    n_imp  = imp.size().getInfo()
    n_keep = min(n_perv, n_imp)
    perv_bal = perv.randomColumn('rand', seed).sort('rand').limit(n_keep)
    imp_bal  = imp.randomColumn('rand', seed).sort('rand').limit(n_keep)
    balanced = perv_bal.merge(imp_bal)

    # 8b) Splitting balanced training dataset into train / validation.
    training   = balanced.filter(ee.Filter.lt('random', 0.75))
    validation = balanced.filter(ee.Filter.gte('random', 0.75))

    print(f"{year_label} Balanced samples: {n_keep} per class, total {balanced.size().getInfo()}")

    # Calling various information from the train/validation dataset to ensure filtering is working correctly.
    n_total       = balanced.size().getInfo()
    n_train       = training.size().getInfo()
    n_validation  = validation.size().getInfo()
    n_pervious    = balanced.filter(ee.Filter.eq('CID', 0)).size().getInfo()
    n_impervious  = balanced.filter(ee.Filter.eq('CID', 1)).size().getInfo()

    print(f"{year_label} total samples = {n_total}, "
          f"training = {n_train}, validation = {n_validation}")
    print(f"{year_label} pervious samples = {n_pervious}, "
          f"impervious samples = {n_impervious}")


    # 9) Hyperparameter fine tuning
    if gamma_values is None:
        gamma_values = [i/10. for i in range(1,11)]
    if cost_values is None:
        cost_values  = list(range(1,11))

    best = {'accuracy': -1}
    for γ in gamma_values:
        for C in cost_values:
            clf = ee.Classifier.libsvm(kernelType='RBF', gamma=γ, cost=C)
            trained = clf.train(training, 'CID', bands)
            acc = (validation
                   .classify(trained)
                   .errorMatrix('CID','classification')
                   .accuracy()
                   .getInfo())
            if acc > best['accuracy']:
                best = {'gamma': γ, 'cost': C, 'accuracy': acc}

    print(f"{year_label} Best hyperparameters γ={best['gamma']}, C={best['cost']}, "
          f"Accuracy={best['accuracy']*100:.1f}%")

    # 10) Train the final classifier and then classifying the image.
    final_clf = ee.Classifier.libsvm(
        kernelType='RBF',
        gamma=best['gamma'],
        cost= best['cost']
    ).train(training, 'CID', bands)

    classified = image.select(bands).classify(final_clf)

    # 11) Compute area stats
    imperv_area = classified.eq(1).multiply(ee.Image.pixelArea())
    perv_area   = classified.eq(0).multiply(ee.Image.pixelArea())

    sum_imp = imperv_area.reduceRegion(
        reducer=ee.Reducer.sum(),
        geometry=catchment.geometry(),
        scale=30,
        maxPixels=1e13
    ).get('classification')

    sum_per = perv_area.reduceRegion(
        reducer=ee.Reducer.sum(),
        geometry=catchment.geometry(),
        scale=30,
        maxPixels=1e13
    ).get('classification')

    total = catchment.geometry().area()

    imp_m2 = sum_imp.getInfo()
    per_m2 = sum_per.getInfo()
    tot_m2 = total.getInfo()

    imp_pct = (imp_m2 / tot_m2) * 100

    return {
        'classifier':      final_clf,
        'classifiedImage': classified,
        'validation':     validation,
        'training':       training,
        'gamma':           best['gamma'],
        'cost':            best['cost'],
        'accuracy':        best['accuracy'],
        'areas_m2': {
            'impervious': imp_m2,
            'pervious':   per_m2,
            'total':      tot_m2
        },
        'impervious_pct':  imp_pct
    }




In [ ]:
bands_L8 = ['SR_B1','SR_B2','SR_B3','SR_B4','SR_B5','SR_B6','SR_B7']
bands_L5 = ['SR_B1','SR_B2','SR_B3','SR_B4','SR_B5','SR_B7']
viz     = {'min':0,'max':1,'palette':['#4caf50','#ff0000']}

# 2024 (no b1_max filtering due to using all stages of impervious cover expansion; Landsat 9 with 7 bands)
res2024 = classifyImperviousSVM(
    image=image4,
    catchment=catchment,
    bands=bands_L8,
    year_label='2024',
    sample_points=700,
    seed=999,
    b1_max=None
)

# 2013 (filter b1 ≤ 7 = up to the year 2015; Landsat 8 with 7 bands)
res2013 = classifyImperviousSVM(
    image=image3,
    catchment=catchment,
    bands=bands_L8,
    year_label='2013',
    sample_points=700,
    seed=999,
    b1_max=7
)

# 1997 (filter b1 ≤ 4 = up to the year 2000; Landsat 5 with 6 bands)
res1997 = classifyImperviousSVM(
    image=image2,
    catchment=catchment,
    bands=bands_L5,
    year_label='1997',
    sample_points=600,
    seed=999,
    b1_max=4
)

# 1984 (filter b1 = 1, only including pre-1985 impervious surfaces; Landsat 5 with 6 bands)
res1984 = classifyImperviousSVM(
    image=image1,
    catchment=catchment,
    bands=bands_L5,
    year_label='1984',
    sample_points=600,
    seed=999,
    b1_max=1
)


In [ ]:
# Print % impervious surface cover.
print(f"1984 impervious surface percentage = {res1984['impervious_pct']:.2f}% of catchment")
print(f"1997 impervious surface percentage = {res1997['impervious_pct']:.2f}% of catchment")
print(f"2013 impervious surface percentage = {res2013['impervious_pct']:.2f}% of catchment")
print(f"2024 impervious surface percentage = {res2024['impervious_pct']:.2f}% of catchment")

# Print impervious surface cover in absolute numbers
print(f"1984 impervious surface area = {res1984['areas_m2']['impervious']/1e6:.2f} km²")
print(f"1997 impervious surface area = {res1997['areas_m2']['impervious']/1e6:.2f} km²")
print(f"2013 impervious surface area = {res2013['areas_m2']['impervious']/1e6:.2f} km²")
print(f"2024 impervious surface area = {res2024['areas_m2']['impervious']/1e6:.2f} km²")

print(f"1984 impervious surface area = {res1984['areas_m2']['total']/1e6:.2f} km²")
print(f"1997 impervious surface area = {res1997['areas_m2']['total']/1e6:.2f} km²")
print(f"2013 impervious surface area = {res2013['areas_m2']['total']/1e6:.2f} km²")
print(f"2024 impervious surface area = {res2024['areas_m2']['total']/1e6:.2f} km²")

In [ ]:
# Creating a map centered on the catchment.
center = catchment.geometry().centroid().coordinates().getInfo()  # [lon, lat]
m = geemap.Map(center=map_center, zoom=12)

# Visual parameters.
viz = {'min': 0, 'max': 1, 'palette': ['#c8d9c5', '#ff0000']}

# Adding each classified result as its own layer.
m.addLayer(res1984['classifiedImage'], viz, 'Impervious 1984')
m.addLayer(res1997['classifiedImage'], viz, 'Impervious 1997')
m.addLayer(res2013['classifiedImage'], viz, 'Impervious 2013')
m.addLayer(res2024['classifiedImage'], viz, 'Impervious 2024')
m.addLayerControl()
m

In [ ]:
# Calculating Change Detection (gain, loss, and net) across all three time ranges with at least 2 steps.

# Defining total area and pixel area for later use.
total_m2  = catchment.geometry().area().getInfo()      # whole catchment
total_km2 = total_m2 / 1e6                             # just for context
pixel_area = ee.Image.pixelArea()                      # re-use later

# From 1984 to 20204
# Defining pixel value +1 = gain; -1 = loss.
change_84_24 = res2024['classifiedImage'].subtract(res1984['classifiedImage'])
gain_84_24   = change_84_24.eq(1)
loss_84_24   = change_84_24.eq(-1)

# Calculating first gain and then loss areas (in square meters).
gain_m2_84_24 = gain_84_24.multiply(pixel_area) \
    .reduceRegion(ee.Reducer.sum(),
                  geometry=catchment.geometry(),
                  scale=30,
                  maxPixels=1e13) \
    .get('classification').getInfo()

loss_m2_84_24 = loss_84_24.multiply(pixel_area) \
    .reduceRegion(ee.Reducer.sum(),
                  geometry=catchment.geometry(),
                  scale=30,
                  maxPixels=1e13) \
    .get('classification').getInfo()

# Converting calculation results to square kilometers.
gain_km2_84_24 = gain_m2_84_24 / 1e6
loss_km2_84_24 = loss_m2_84_24 / 1e6
net_km2_84_24  = (gain_m2_84_24 - loss_m2_84_24) / 1e6

# Converting calculation results to a percentage point increase/decrease.
gain_pct_84_24 = (gain_m2_84_24 / total_m2) * 100
loss_pct_84_24 = (loss_m2_84_24 / total_m2) * 100
net_pct_84_24  = ((gain_m2_84_24 - loss_m2_84_24) / total_m2) * 100

print(f"1984-2024: Gain {gain_km2_84_24:.2f} km2 ({gain_pct_84_24:.2f} %), "
      f"Loss {loss_km2_84_24:.2f} km2 (-{loss_pct_84_24:.2f} %), "
      f"Net {net_km2_84_24:.2f} km2 ({net_pct_84_24:.2f} %)")


#####
# Same process for 1984 to 2013.
change_84_13 = res2013['classifiedImage'].subtract(res1984['classifiedImage'])
gain_84_13   = change_84_13.eq(1)
loss_84_13   = change_84_13.eq(-1)

gain_m2_84_13 = gain_84_13.multiply(pixel_area) \
    .reduceRegion(ee.Reducer.sum(),
                  geometry=catchment.geometry(),
                  scale=30,
                  maxPixels=1e13) \
    .get('classification').getInfo()

loss_m2_84_13 = loss_84_13.multiply(pixel_area) \
    .reduceRegion(ee.Reducer.sum(),
                  geometry=catchment.geometry(),
                  scale=30,
                  maxPixels=1e13) \
    .get('classification').getInfo()

gain_km2_84_13 = gain_m2_84_13 / 1e6
loss_km2_84_13 = loss_m2_84_13 / 1e6
net_km2_84_13  = (gain_m2_84_13 - loss_m2_84_13) / 1e6

gain_pct_84_13 = (gain_m2_84_13 / total_m2) * 100
loss_pct_84_13 = (loss_m2_84_13 / total_m2) * 100
net_pct_84_13  = ((gain_m2_84_13 - loss_m2_84_13) / total_m2) * 100

print(f"1984-2013: Gain {gain_km2_84_13:.2f} km2 ({gain_pct_84_13:.2f} %), "
      f"Loss {loss_km2_84_13:.2f} km2 (-{loss_pct_84_13:.2f} %), "
      f"Net {net_km2_84_13:.2f} km2 ({net_pct_84_13:.2f} %)")


#####
# And for 1997 to 2024.
change_97_24 = res2024['classifiedImage'].subtract(res1997['classifiedImage'])
gain_97_24   = change_97_24.eq(1)
loss_97_24   = change_97_24.eq(-1)

gain_m2_97_24 = gain_97_24.multiply(pixel_area) \
    .reduceRegion(ee.Reducer.sum(),
                  geometry=catchment.geometry(),
                  scale=30,
                  maxPixels=1e13) \
    .get('classification').getInfo()

loss_m2_97_24 = loss_97_24.multiply(pixel_area) \
    .reduceRegion(ee.Reducer.sum(),
                  geometry=catchment.geometry(),
                  scale=30,
                  maxPixels=1e13) \
    .get('classification').getInfo()

gain_km2_97_24 = gain_m2_97_24 / 1e6
loss_km2_97_24 = loss_m2_97_24 / 1e6
net_km2_97_24  = (gain_m2_97_24 - loss_m2_97_24) / 1e6

gain_pct_97_24 = (gain_m2_97_24 / total_m2) * 100
loss_pct_97_24 = (loss_m2_97_24 / total_m2) * 100
net_pct_97_24  = ((gain_m2_97_24 - loss_m2_97_24) / total_m2) * 100

print(f"1997-2024: Gain {gain_km2_97_24:.2f} km2 ({gain_pct_97_24:.2f} %), "
      f"Loss {loss_km2_97_24:.2f} km2 (-{loss_pct_97_24:.2f} %), "
      f"Net {net_km2_97_24:.2f} km2 ({net_pct_97_24:.2f} %)")

In [ ]:
# Confusion matrix-based validation using Overall Accuracy, Producer's Accuracy,
# User's Accuarcy, Kappa Coefficient, and F1-Score.

# image1 (1984)
validated1 = res1984['validation'].classify(res1984['classifier'])

confmatrix1 = validated1.errorMatrix('CID', 'classification')

print('1984 Confusion Matrix', confmatrix1.getInfo())

print('1984 Overall Accuracy:', confmatrix1.accuracy().getInfo())
print('1984 Producers Accuracy:', confmatrix1.producersAccuracy().getInfo())
print('1984 Users Accuracy:', confmatrix1.consumersAccuracy().getInfo())
print('1984 Kappa Coefficient:', confmatrix1.kappa().getInfo())
print('1984 F1-Score:', confmatrix1.fscore(1).getInfo())

# image2 (1997)
validated2 = res1997['validation'].classify(res1997['classifier'])

confmatrix2 = validated2.errorMatrix('CID', 'classification')

print('1997 Confusion Matrix', confmatrix2.getInfo())

print('1997 Overall Accuracy:', confmatrix2.accuracy().getInfo())
print('1997 Producers Accuracy:', confmatrix2.producersAccuracy().getInfo())
print('1997 Users Accuracy:', confmatrix2.consumersAccuracy().getInfo())
print('1997 Kappa Coefficient:', confmatrix2.kappa().getInfo())
print('1997 F1-Score:', confmatrix2.fscore(1).getInfo())

# image3 (2013)
validated3 = res2013['validation'].classify(res2013['classifier'])

confmatrix3 = validated3.errorMatrix('CID', 'classification')

print('2013 Confusion Matrix', confmatrix3.getInfo())

print('2013 Overall Accuracy:', confmatrix3.accuracy().getInfo())
print('2013 Producers Accuracy:', confmatrix3.producersAccuracy().getInfo())
print('2013 Users Accuracy:', confmatrix3.consumersAccuracy().getInfo())
print('2013 Kappa Coefficient:', confmatrix3.kappa().getInfo())
print('2013 F1-Score:', confmatrix3.fscore(1).getInfo())

# image4 (2024)
validated4 = res2024['validation'].classify(res2024['classifier'])

confmatrix4 = validated4.errorMatrix('CID', 'classification')

print('2024 Confusion Matrix', confmatrix4.getInfo())

print('2024 Overall Accuracy:', confmatrix4.accuracy().getInfo())
print('2024 Producers Accuracy:', confmatrix4.producersAccuracy().getInfo())
print('2024 Users Accuracy:', confmatrix4.consumersAccuracy().getInfo())
print('2024 Kappa Coefficient:', confmatrix4.kappa().getInfo())
print('2024 F1-Score:', confmatrix4.fscore(1).getInfo())